Assume we already have a Python function that calls OpenAI API

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from llm import my_llm_tool

deployment_name = "text-davinci-003"

# 0. Set up an OpenTelemetry collector

Implement a simple collecotr that print the traces to stdout.

In [ ]:
import threading
from http.server import BaseHTTPRequestHandler, HTTPServer

from opentelemetry.proto.collector.trace.v1.trace_service_pb2 import ExportTraceServiceRequest


class OTLPCollector(BaseHTTPRequestHandler):
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)

        traces_request = ExportTraceServiceRequest()
        traces_request.ParseFromString(post_data)
        
        print("Received a POST request with data:")
        print(traces_request)

        self.send_response(200, "Traces received")
        self.end_headers()
        self.wfile.write(b'Data received and printed to stdout.\n')

def run_server(port: int):
    server_address = ('', port)
    httpd = HTTPServer(server_address, OTLPCollector)
    httpd.serve_forever()

def start_server(port: int):
    server_thread = threading.Thread(target=run_server, args=(port,))
    server_thread.daemon = True
    server_thread.start()
    print(f"Server started on port {port}. Access http://localhost:{port}/")
    return server_thread

In [ ]:
# invoke the collecotr service, serving on OTLP port
start_server(port=4318)

# 1. Trace your application with tracing

Call `start_trace()`, and configure the OTLP exporter to above collector.

In [ ]:
from promptflow.tracing import start_trace

start_trace()

In [ ]:
from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import BatchSpanProcessor

In [ ]:
tracer_provider = trace.get_tracer_provider()
otlp_span_exporter = OTLPSpanExporter()
tracer_provider.add_span_processor(BatchSpanProcessor(otlp_span_exporter))

Visualize traces in the stdout.

In [ ]:
result = my_llm_tool(
    prompt="Write a simple Hello, world! program that displays the greeting message when executed.",
    deployment_name=deployment_name,
)
result
# view the traces under this cell